In [1]:
### UTILS
def get_input(dayNum):
    import os
    data_file = f"./data/day-{dayNum}.input"
    if not os.path.exists(data_file):
        download_input(dayNum) 
    return [line.strip() for line in open(data_file).readlines()]

def download_input(dayNum):
    import subprocess
    SESSION_COOKIE = open('./session_cookie.secret').readlines()[0].strip()
    url = f"https://adventofcode.com/2020/day/{dayNum}/input"
    cmd = f"curl -H 'Cookie: session={SESSION_COOKIE}' {url} > ./data/day-{dayNum}.input"
    subprocess.run(cmd, capture_output=True, shell=True)

# input is e.g. [0,1,0,0]
# output is e.g 0b0100 -> 4
def bits_to_int(bits):
    return int("0b" + "".join([str(b) for b in bits]), base=2)

assert(bits_to_int([0,1,0,0]) == 4)

In [2]:
# https://adventofcode.com/2020/day/4
def Day4():
    def get_passports():
        passports = []
        lines = get_input(4)
        cur_passport = {}
        for line in lines:
            if line == "":
                if cur_passport:
                    passports.append(cur_passport)
                cur_passport = {}
            field_pairs = line.split()
            for pair in field_pairs:
                [key,val] = pair.split(':')
                cur_passport[key] = val
        passports.append(cur_passport)
        return passports
        
    passports = get_passports()
    
    def valid_1(passport):
        required_fields = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']
        if not all([key in passport for key in required_fields]):
            return False
        return True

    def valid_2(passport):
        import re
        if not valid_1(passport):
            return False

        def validate_height(x):
            match = re.match("(\d+)(cm|in)", x)
            if match is None:
                return False
            [hgt,unit] = match.groups()
            hgt = int(hgt)
            if unit == 'cm':
                return hgt >= 150 and hgt <= 193
            elif unit == 'in':
                return hgt >= 59 and hgt <= 76
            else:
                raise f"Unexpected value for hgt {hgt} unit {unit}"
        
        validations = {
            'byr': lambda x: int(x) >= 1920 and int(x) <= 2002,
            'iyr': lambda x: int(x) >= 2010 and int(x) <= 2020,
            'eyr': lambda x: int(x) >= 2020 and int(x) <= 2030,
            'hgt': validate_height,
            'hcl': lambda x: re.match("^#[0-9a-f]{6}$",x) is not None,
            'ecl': lambda x: x in 'amb blu brn gry grn hzl oth'.split(' '),
            'pid': lambda x: re.match("^[0-9]{9}$",x) is not None,
        }
        
        for field in passport:
            if field in validations:
                if not validations[field](passport[field]):
                    return False
        return True

    def day1():
        return len([p for p in passports if valid_1(p)])
    
    def day2():
        return len([p for p in passports if valid_2(p)])
    
    return day1(), day2()

print(f"Day 4: {Day4()}")

Day 4: (190, 121)


In [4]:
# https://adventofcode.com/2020/day/5
def Day5():
    def pid(d):
        row_bits = [1 if b=='B' else 0 for b in d[:7]]
        col_bits = [1 if b=='R' else 0 for b in d[7:]]
        row = bits_to_int(row_bits)
        col = bits_to_int(col_bits)
        return row*8 + col
    
    def pids():
        return [pid(d) for d in get_input(5)]
    
    def part1():
        return max(pids())
    
    def part2():
        pids_ = pids()
        for id in sorted(pids_):
            if (id+1) not in pids_:
                return id + 1
            
    return part1(), part2()
    
print(f"Day 5: {Day5()}")

Day 5: (976, 685)


In [33]:
def Day6():
    data = get_input(6)
    
    def part1():
        groups = []
        group = set()
        for line in data:
            if line == '':
                groups.append(group)
                group = set()
            else:
                for char in line:
                    group.add(char)
            
        if group is not None:
            groups.append(group)
            
        return sum([len(g) for g in groups])
            
    def part2():
        groups = []
        group = {'len': 0}
        for line in data:
            if line == '':
                groups.append(group)
                group = {'len':0}
            else:
                group['len'] += 1
                for char in line:
                    if char not in group:
                        group[char] = 1
                    else:
                        group[char] += 1
        if group is not None:
            groups.append(group)
        
        count = 0
        for group in groups:
            g_len = group['len']
            del group['len']
            count += len([k for k in group if group[k] == g_len])
        return count
            
    
    return part1(), part2()

print(f"Day 6: {Day6()}")

Day 6: (6430, 3125)
